In [30]:
#"impoting required packages"
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
from IPython.display import clear_output
import random
import selenium
from selenium.common.exceptions import NoSuchElementException

In [31]:
#header to access the links
header={'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36'}

In [32]:
#Initialising selenium
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')
browser = webdriver.Chrome(executable_path=r'C:\Users\Admin\Desktop\chromedriver.exe')

In [33]:
#reading csv file
#naukri_link = (r"C:\Users\Admin\Desktop\Pytorch\ISB_Assignment\link_by_areas.csv") 
naukri_link = pd.read_csv('link_by_areas.csv')

In [34]:
#to get the links from which the required o/p is to be generated
soupList = list()
for weblink in range(0,len(naukri_link)): 
    browser.get(naukri_link['link'][weblink])
    job_soup=BeautifulSoup(browser.page_source, 'lxml')#returns html of the page
    for info_card in job_soup.findAll(attrs={'class':"jobTuple bgWhite br4 mb-8"}):
        for info in info_card.findAll(attrs={'class':"title fw500 ellipsis"}):
            soupList.append(info.get('href')) #stores all the link of the job postings

In [49]:
Salary = [] 
Experience = []
Location = []
Description = [] 
Role = [] 
Type_of_Hiring = [] 
Industry_Classification = [] 
Qualifications = [] 
Key_Skills = [] 
Company_Description = [] 
Role_Tags = [] 
Scope  = []

In [50]:
for job_link in range(len(soupList)):
    browser.get(soupList[job_link])
    jobSoup=BeautifulSoup(browser.page_source, 'lxml')
    if jobSoup.find(attrs={'class':"salary"})==None:
        continue
    else:
        if jobSoup.find(attrs={'class':"exp"})==None :
            Experience.append('Not mentioned')
        else:
            Experience.append(jobSoup.find(attrs={'class':"exp"}).text)
        Salary.append(jobSoup.find(attrs={'class':"salary"}).text)
        if jobSoup.find(attrs={'class':'loc'})==None :
            Location.append('Not mentioned')
        else:
            Location.append(jobSoup.find(attrs={'class':'loc'}).find('a').text)

        
        Description.append(jobSoup.find(attrs={'class':"job-desc"}).text)
            

        other_tags = []

        for data in jobSoup.find(attrs={'class':"other-details"}).findAll(attrs={'class':"details"}):
            other_tags.append(data.text)
        other_tags[0].replace('Role', ' ')
        Role.append(other_tags[0])
        other_tags[1].replace('Industry', ' ').replace('Type', ' ')
        Industry_Classification.append(other_tags[1])
        other_tags[2].replace('Functional', ' ').replace('Area', ' ')
        Scope.append(other_tags[2])
        other_tags[3].replace('Employment', ' ').replace('Type', ' ')
        Type_of_Hiring.append(other_tags[3])
        other_tags[4].replace('Role', ' ').replace('Category', ' ')
        Role_Tags.append(other_tags[4])

        eligibility=[]
        
        for tag in jobSoup.find(attrs={'class':"education"}).findAll(attrs={'class':'details'}):
            eligibility.append(tag.text)
        Qualifications.append(eligibility)
        
        key=[]
        
        if jobSoup.find(attrs={'class':"key-skill"})==None:
            key.append('No_value')
            Key_Skills.append(key)
        else :
            for keyskill in jobSoup.find(attrs={'class':"key-skill"}).findAll('a'):
                key.append(keyskill.text)
            Key_Skills.append(key)
        if jobSoup.find(attrs={'class':"about-company"}).find(attrs={'class':"detail dang-inner-html"})==None:
            Company_Description.append('No_value')
            #else :
                #company.append(soup.find(attrs={'class':"about-company"}.text)
        else :                       
            Company_Description.append(jobSoup.find(attrs={'class':"about-company"}).find(attrs={'class':"detail dang-inner-html"}).text)

In [55]:
dfs=pd.DataFrame()

In [59]:
#Transferring the data to a data-frame
dfs['Company_Description'] = Company_Description
dfs['Salary'] = Salary
dfs['Experience'] = Experience
dfs['Location'] = Location
dfs['Description'] = Description
dfs['Key_Skills'] = Key_Skills
dfs['Qualifications'] = Qualifications
dfs['Type_of_Hiring']= Type_of_Hiring
dfs['Industry_Classification'] = Industry_Classification
dfs['Scope'] = Scope
dfs['Role_Tags'] = Role_Tags

In [60]:
dfs.head()

,Salary,Experience,Location,Description,Key_Skills,Qualifications,Type_of_Hiring,Industry_Classification,Scope,Role_Tags,Company_Description
0,Not Disclosed,0 - 3 years,Mumbai,Job description Job Description Minimum Qua...,"[Tally, Journal entries, Finance, Business adv...","[UG :Any Graduate in Any Specialization, PG :P...","Employment TypeFull Time, Permanent","Industry TypeIT-Software, Software Services,","Functional AreaAccounts, Finance, Tax, Company...",Role CategoryAccounts,KaryaMitr is born with a vision to enable huma...
1,"₹ 1,75,000 - 3,00,000 P.A.",2 - 5 years,Kolkata,Job descriptionAre you an Accounting Super Her...,"[Debtors, Tally, cash flow statement, accounts...","[UG :B.Com in Commerce, PG :CA in CA, Second A...","Employment TypeFull Time, Permanent","Industry TypeAccounting, Finance,","Functional AreaAccounts, Finance, Tax, Company...",Role CategoryAccounts,Source Desk Global is a global service provide...
2,"₹ 18,00,000 - 27,50,000 P.A.",1 - 6 years,Canada,"Job descriptionWork with accounts payable, acc...","[accounts receivable, accounts payable, paymen...","[UG :Graduation Not Required, Any Graduate in ...","Employment TypeFull Time, Permanent","Industry TypeMedical, Healthcare, Hospitals,","Functional AreaAccounts, Finance, Tax, Company...",Role CategoryAccounts,Appreciate it if you could revert back with yo...
3,Not Disclosed,1 - 2 years,Hyderabad,Job descriptionDescription Who We Are B...,"[Wireless, Payroll, Journal entries, Financial...","[UG :B.Com in Commerce, Any Graduate in Any Sp...","Employment TypeFull Time, Permanent","Industry TypeIT-Software, Software Services,","Functional AreaAccounts, Finance, Tax, Company...",Role CategoryAccounts,Bridge International Academies
4,"₹ 5,00,000 - 8,00,000 P.A.",4 - 9 years,Delhi NCR,Job descriptionIntercompany AccountingMonth En...,"[Intercompany Accounting, General Accounting]","[UG :B.Com in Commerce, PG :Any Postgraduate i...","Employment TypeFull Time, Permanent","Industry TypeAccounting, Finance,","Functional AreaAccounts, Finance, Tax, Company...",Role CategoryAccounts,A reputed U.S. Multinational Company engaged i...


In [61]:
dfs.to_csv('ISB_Task2.csv',index=False)